***IMPORT DES DONNEES DE TRAFIC, BRUIT, METEO ET SONDAGE***

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\PPBE_BdxMet\src\Carac_situ_acoustiq')
sys.path.append(r'C:\Users\martin.schoreisz\git\PPBE_BdxMet\src')

import os 
from datetime import datetime
import pandas as pd
import Connexion_Transfert as ct
from Import_stockage_donnees.Import_export_bdd import FichierMesureBruitCsv

# DONNEES BRUIT

## Lecture des fichiers ods

In [12]:
# test de formatage sur un fichier
dossier = r'C:\Users\martin.schoreisz\Documents\temp\BruitRessenti\mesures\CSV\sono1_10RuePierreRonsard'
nomFichier = 'sono1_29-03-2022.csv'
cheminFichier = os.path.join(dossier, nomFichier)

In [9]:
fichier = FichierMesureBruitCsv(cheminFichier)

In [33]:
listeAPasser = ['sono1_01-04-2022.csv', 'sono1_02-04-2022.csv', 'sono1_03-04-2022.csv', 'sono1_04-04-2022.csv', 'sono1_21-03-2022.csv', 'sono1_28-03-2022.csv',
                'sono1_22-03-2022.csv', 'sono1_23-03-2022.csv', 'sono1_24-03-2022.csv', 'sono1_25-03-2022.csv', 'sono1_26-03-2022.csv', 'sono1_27-03-2022.csv']
with ct.ConnexionBdd('ech24') as c:
    for r, d, files in os.walk(r'C:\Users\martin.schoreisz\Documents\temp\BruitRessenti\mesures\CSV\sono4_26RueFrancoisVillon'):
        for f in files:
            if f in listeAPasser:
                continue
            if f .endswith('.csv'):
                cheminFichier = os.path.join(r, f)
                timerAvantCalcul = datetime.now()
                print(f"fichier {cheminFichier}, avant calcul {datetime.now()}")
                fichier = FichierMesureBruitCsv(cheminFichier)
                timerApresCalcul = datetime.now()
                print(f"duree calcul : {timerApresCalcul-timerAvantCalcul}")
                fichier.dfNiveauSpectre.to_sql(schema='mesures_physiques', name='niveau_bruit', con=c.sqlAlchemyConn, if_exists='append', index=False)
                timerApresNiveaux = datetime.now()
                print(f"duree transfert spectre {timerApresNiveaux-timerApresCalcul}")

fichier C:\Users\martin.schoreisz\Documents\temp\BruitRessenti\mesures\CSV\sono4_26RueFrancoisVillon\sono4_04-04-2022.csv, avant calcul 2022-06-08 14:44:18.511738
duree calcul : 0:00:07.657050
duree transfert spectre 0:00:31.493580
fichier C:\Users\martin.schoreisz\Documents\temp\BruitRessenti\mesures\CSV\sono4_26RueFrancoisVillon\sono4_05-04-2022.csv, avant calcul 2022-06-08 14:44:57.663368
duree calcul : 0:00:19.580888
duree transfert spectre 0:01:22.676149
fichier C:\Users\martin.schoreisz\Documents\temp\BruitRessenti\mesures\CSV\sono4_26RueFrancoisVillon\sono4_06-04-2022.csv, avant calcul 2022-06-08 14:46:39.920405
duree calcul : 0:00:18.995448
duree transfert spectre 0:01:22.311128
fichier C:\Users\martin.schoreisz\Documents\temp\BruitRessenti\mesures\CSV\sono4_26RueFrancoisVillon\sono4_07-04-2022.csv, avant calcul 2022-06-08 14:48:21.227981
duree calcul : 0:00:19.500105
duree transfert spectre 0:01:21.216466
fichier C:\Users\martin.schoreisz\Documents\temp\BruitRessenti\mesures\C

In [29]:
test.loc[(test == 0).any(axis=1)]

,date_heure,leq_a,leq_lin,crete_c,fast_inst_a,fast_max_a,fast_min_a,to_6,to_8,to_10,...,to_2500,to_3150,to_4000,to_5000,to_6300,to_8000,to_10000,to_12500,to_16000,to_20000
55820,2022-03-29 01:56:50.500,47.6,58.0,65.2,47.7,48.0,47.6,0.0,34.6,43.8,...,16.9,11.4,8.2,7.5,8.7,7.9,8.5,9.9,10.2,10.9
149173,2022-03-29 05:11:19.625,50.7,58.8,68.8,50.8,51.4,50.7,0.0,43.9,40.5,...,29.4,25.4,20.5,17.2,14.5,10.4,9.0,9.7,10.1,10.9
284712,2022-03-29 09:53:42.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
284713,2022-03-29 09:53:42.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
284714,2022-03-29 09:53:42.250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285771,2022-03-29 09:55:54.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
285772,2022-03-29 09:55:54.500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
285773,2022-03-29 09:55:54.625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
285774,2022-03-29 09:55:54.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
